https://www.denvergov.org/opendata/dataset/city-and-county-of-denver-crime-marijuana

crime in denver related to marijuana
    

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
crime_den = pd.read_csv('crime_Denver.csv')

In [4]:
crime_den.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464110 entries, 0 to 464109
Data columns (total 19 columns):
INCIDENT_ID               464110 non-null int64
OFFENSE_ID                464110 non-null int64
OFFENSE_CODE              464110 non-null int64
OFFENSE_CODE_EXTENSION    464110 non-null int64
OFFENSE_TYPE_ID           464110 non-null object
OFFENSE_CATEGORY_ID       464110 non-null object
FIRST_OCCURRENCE_DATE     464110 non-null object
LAST_OCCURRENCE_DATE      146850 non-null object
REPORTED_DATE             464110 non-null object
INCIDENT_ADDRESS          418457 non-null object
GEO_X                     460330 non-null float64
GEO_Y                     460330 non-null float64
GEO_LON                   460330 non-null float64
GEO_LAT                   460330 non-null float64
DISTRICT_ID               464110 non-null int64
PRECINCT_ID               464110 non-null int64
NEIGHBORHOOD_ID           464110 non-null object
IS_CRIME                  464110 non-null int64
IS_TRAFFIC

In [5]:
# we see a bunch of NaN's in last occurrence date, really for 3/4 of the entries.  
# most other entries look well documented.  
# we see a few NaN's for addresses as well as geo coordinates - maybe we drop address and just look at geo for our heat map idea.

In [6]:
crime_den.describe()

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,IS_CRIME,IS_TRAFFIC
count,4.641100e+05,4.641100e+05,464110.000000,464110.000000,4.603300e+05,4.603300e+05,460330.000000,460330.000000,464110.000000,464110.000000,464110.000000,464110.000000
mean,3.769763e+09,3.769763e+15,3841.337088,0.204303,3.162980e+06,1.695046e+06,-104.929643,39.719571,3.484499,365.931734,0.739738,0.260781
std,9.458898e+09,9.458898e+15,1655.943928,0.577768,6.728593e+05,1.656075e+05,1.883052,0.713067,1.770930,177.810837,0.438778,0.439061
min,2.015800e+04,2.015854e+10,902.000000,0.000000,3.050128e+06,1.648935e+06,-105.321797,0.000001,1.000000,111.000000,0.000000,0.000000
25%,2.014628e+09,2.014628e+15,2305.000000,0.000000,3.138101e+06,1.683030e+06,-105.009021,39.707511,2.000000,213.000000,0.000000,0.000000
50%,2.016511e+09,2.016511e+15,3572.000000,0.000000,3.145543e+06,1.694757e+06,-104.982601,39.739773,3.000000,323.000000,1.000000,0.000000
75%,2.018217e+09,2.018217e+15,5441.000000,0.000000,3.161707e+06,1.701579e+06,-104.925442,39.758450,5.000000,521.000000,1.000000,1.000000
max,2.018792e+12,2.018792e+18,7399.000000,5.000000,4.067476e+07,1.141836e+07,-0.000001,39.904311,7.000000,759.000000,1.000000,1.000000


In [7]:
crime_den.head()

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC
0,2016376978,2016376978521300,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,NaN,3193983.0,1707251.0,-104.809881,39.773188,5,521,montbello,1,0
1,20186000994,20186000994239900,2399,0,theft-other,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,NaN,3201943.0,1711852.0,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0
2,20166003953,20166003953230500,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,2932 S JOSEPHINE ST,3152762.0,1667011.0,-104.957381,39.663490,3,314,wellshire,1,0
3,201872333,201872333239900,2399,0,theft-other,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,705 S COLORADO BLVD,3157162.0,1681320.0,-104.941440,39.702698,3,312,belcaro,1,0
4,2017411405,2017411405230300,2303,0,theft-shoplift,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,2810 E 1ST AVE,3153211.0,1686545.0,-104.955370,39.717107,3,311,cherry-creek,1,0


In [8]:
crime_types = crime_den['OFFENSE_TYPE_ID'].unique()
len(crime_types)
# there are 198 different kinds of crime

198

In [9]:
type(crime_den['FIRST_OCCURRENCE_DATE'][0])
# no good we need this in datetime

str

In [10]:
date_series = crime_den['FIRST_OCCURRENCE_DATE']
date_occurred = pd.to_datetime(date_series)
crime_den['date_occurred'] = date_occurred



In [11]:
crime_den = crime_den.drop('FIRST_OCCURRENCE_DATE', axis=1)

In [14]:
crime_den['NEIGHBORHOOD_ID'].unique()
# no NaN's!  we'll use this for area/regional analysis.

array(['montbello', 'gateway-green-valley-ranch', 'wellshire', 'belcaro',
       'cherry-creek', 'city-park-west', 'villa-park', 'speer',
       'civic-center', 'highland', 'goldsmith', 'ruby-hill',
       'cory-merrill', 'jefferson-park', 'windsor', 'lincoln-park',
       'washington-park-west', 'stapleton', 'lowry-field', 'five-points',
       'elyria-swansea', 'hampden-south', 'dia', 'platt-park',
       'west-colfax', 'harvey-park-south', 'baker', 'cheesman-park',
       'cbd', 'rosedale', 'hilltop', 'barnum-west', 'virginia-village',
       'west-highland', 'chaffee-park', 'north-park-hill',
       'washington-virginia-vale', 'barnum', 'washington-park',
       'sunnyside', 'westwood', 'northeast-park-hill', 'harvey-park',
       'country-club', 'sun-valley', 'university-hills', 'clayton',
       'east-colfax', 'congress-park', 'hampden', 'globeville',
       'north-capitol-hill', 'berkeley', 'auraria', 'sloan-lake',
       'university', 'university-park', 'union-station', 'mar-le

In [12]:
crime_den.head()

# incident ID and Offense ID most likely are not useful for our analysis.
# ditto with offense_code_extension, last occurrence date, reported date, all geo's, district ID and precinct id

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,date_occurred
0,2016376978,2016376978521300,5213,0,weapon-unlawful-discharge-of,all-other-crimes,NaN,6/15/2016 11:31:00 PM,NaN,3193983.0,1707251.0,-104.809881,39.773188,5,521,montbello,1,0,2016-06-15 23:31:00
1,20186000994,20186000994239900,2399,0,theft-other,larceny,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,NaN,3201943.0,1711852.0,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0,2017-10-11 12:30:00
2,20166003953,20166003953230500,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,2932 S JOSEPHINE ST,3152762.0,1667011.0,-104.957381,39.663490,3,314,wellshire,1,0,2016-03-04 20:00:00
3,201872333,201872333239900,2399,0,theft-other,larceny,NaN,1/30/2018 10:29:00 PM,705 S COLORADO BLVD,3157162.0,1681320.0,-104.941440,39.702698,3,312,belcaro,1,0,2018-01-30 19:20:00
4,2017411405,2017411405230300,2303,0,theft-shoplift,larceny,NaN,6/23/2017 4:09:00 PM,2810 E 1ST AVE,3153211.0,1686545.0,-104.955370,39.717107,3,311,cherry-creek,1,0,2017-06-22 20:53:00


In [15]:
crime_den = crime_den.drop(['INCIDENT_ID', 'OFFENSE_ID', 'OFFENSE_CODE_EXTENSION', 'OFFENSE_CODE', 'LAST_OCCURRENCE_DATE', 'REPORTED_DATE', 'INCIDENT_ADDRESS', 'GEO_X' ,'GEO_Y', 'GEO_LON', 'GEO_LAT', 'DISTRICT_ID', 'PRECINCT_ID', 'IS_CRIME', 'IS_TRAFFIC'],axis=1)
crime_den.head()

,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,NEIGHBORHOOD_ID,date_occurred
0,weapon-unlawful-discharge-of,all-other-crimes,montbello,2016-06-15 23:31:00
1,theft-other,larceny,gateway-green-valley-ranch,2017-10-11 12:30:00
2,theft-items-from-vehicle,theft-from-motor-vehicle,wellshire,2016-03-04 20:00:00
3,theft-other,larceny,belcaro,2018-01-30 19:20:00
4,theft-shoplift,larceny,cherry-creek,2017-06-22 20:53:00


In [16]:
crime_den.to_csv('clean_den_crime.csv')

In [13]:
category = crime_den['OFFENSE_CATEGORY_ID']
category.unique()

array(['all-other-crimes', 'larceny', 'theft-from-motor-vehicle',
       'traffic-accident', 'drug-alcohol', 'auto-theft',
       'white-collar-crime', 'burglary', 'public-disorder',
       'aggravated-assault', 'other-crimes-against-persons', 'robbery',
       'sexual-assault', 'murder', 'arson'], dtype=object)

In [16]:
crime_types

array(['weapon-unlawful-discharge-of', 'theft-other',
       'theft-items-from-vehicle', 'theft-shoplift', 'traf-other',
       'theft-parts-from-vehicle', 'criminal-trespassing',
       'traffic-accident-hit-and-run', 'drug-methampetamine-possess',
       'theft-of-motor-vehicle', 'fraud-by-use-of-computer',
       'traffic-accident', 'burglary-business-by-force', 'theft-embezzle',
       'threats-to-injure', 'theft-from-bldg', 'liquor-possession',
       'aggravated-assault', 'criminal-mischief-mtr-veh',
       'kidnap-adult-victim', 'violation-of-restraining-order',
       'criminal-mischief-other', 'assault-simple',
       'harassment-stalking-dv', 'traf-vehicular-assault',
       'vehicular-eluding-no-chase', 'robbery-street',
       'public-order-crimes-other', 'drug-synth-narcotic-possess',
       'burglary-residence-by-force', 'burglary-business-no-force',
       'contraband-into-prison', 'criminal-mischief-graffiti',
       'drug-heroin-possess', 'drug-poss-paraphernalia',
   

In [17]:
revenue_den = pd.read_csv('denver_revenue.csv')

In [21]:
revenue_den.tail()
# would love to use a metric that charted from January 2014 or before

,YEAR,MONTH,GROSS_SALES_TYPE,GROSS_SALES
85,2014,MARCH,Medical Total Gross Sales,16537955.80
86,2014,FEBRUARY,Medical Total Gross Sales,15021878.45
87,2014,FEBRUARY,Retail Total Gross Sales,8591699.44
88,2014,JANUARY,Medical Total Gross Sales,15462182.32
89,2014,JANUARY,Retail Total Gross Sales,8490576.55


http://data.opencolorado.org/dataset/city-and-county-of-denver-marijuana-number-of-active-licenses/resource/d48e95a3-7336-42bb-bc50-6c8b33ac3928
    
Active Marijuana Licenses in Denver

In [19]:
licenses_den = pd.read_csv('denver_licenses_by_monthyear.csv')
'''
https://www.denvergov.org/opendata/dataset/city-and-county-of-denver-marijuana-new-licenses-issued

new marijuana licenses issued by month.  Useful to track spread/growth of industry

Laws were passed to legalize recreational Marijuana in 2012, but did not go into effect until 
'''

'\nhttps://www.denvergov.org/opendata/dataset/city-and-county-of-denver-marijuana-new-licenses-issued\n\nnew marijuana licenses issued by month.  Useful to track spread/growth of industry\n'

In [20]:
licenses_den

,MONTH,YEAR,TOTAL_LICENSES_ISSUED
0,April,2013,4
1,August,2013,10
2,December,2013,13
3,February,2013,8
4,January,2013,7
5,July,2013,1
6,June,2013,36
7,March,2013,6
8,May,2013,3
9,November,2013,2


In [20]:
active_lic = pd.read_csv('den_marijuana_active_business_licenses.csv')
active_lic.head()

,Business File Number,License Type,Entity Name,Trade Name,Current License Status,Expiration Date,Facility Street Number,Facility Pre-Direction,Facility Street Name,Facility Street Type,Facility Unit Number,Facility City,Facility Zip Code
0,2013-BFN-1070104,Retail Marijuana Store,"HERBAL ALTERNATIVES, LLC",NEW BROADSTERDAM,License Issued - Active,2/24/2020 12:00:00 AM,2568,S,BROADWAY,NaN,NaN,DENVER,80210.0
1,2014-BFN-1072246,Retail Marijuana Inf Prod Mfg,"GAIAS GARDEN, LLC",GIA'S GARDEN,License Issued - Active,5/21/2019 12:00:00 AM,4785,N,Elati,NaN,11,Denver,80216.0
2,2014-BFN-1072819,Retail Marij Opt. Prem. Cultiv,"PLATTE VALLEY DISPENSARY, LLC",PLATTE VALLEY DISPENSARY,License Issued - Active,9/17/2019 12:00:00 AM,3104,NaN,BLAKE,ST,NaN,DENVER,80205.0
3,2014-BFN-1073783,Retail Marij Opt. Prem. Cultiv,"ABUNDANT HEALTH DENVER, LLC","HERBAN UNDERGROUND, LLC",License Issued - Active,1/22/2020 12:00:00 AM,70,N,BROADWAY,NaN,50,DENVER,80203.0
4,2014-BFN-1074231,Med Marijuana Opt Prem Cultiv,SWEETWATER PARTNERS LLC,WS EL JEFE,License Issued - Active,10/23/2019 12:00:00 AM,4260,NaN,KEARNEY,ST,NaN,DENVER,80216.0


In [24]:
# looking at min and max, this data goes back until 2010 and until 2018.  Should be a better fit with the la licensing data we have.

In [25]:
active_lic = active_lic.drop(['License Type', 'Trade Name', 'Expiration Date', 'Facility Unit Number', 'Facility Street Type', 'Facility Pre-Direction'],axis=1)
active_lic.head()

,Business File Number,Entity Name,Current License Status,Facility Street Number,Facility Street Name,Facility City,Facility Zip Code
0,2013-BFN-1070104,"HERBAL ALTERNATIVES, LLC",License Issued - Active,2568,BROADWAY,DENVER,80210.0
1,2014-BFN-1072246,"GAIAS GARDEN, LLC",License Issued - Active,4785,Elati,Denver,80216.0
2,2014-BFN-1072819,"PLATTE VALLEY DISPENSARY, LLC",License Issued - Active,3104,BLAKE,DENVER,80205.0
3,2014-BFN-1073783,"ABUNDANT HEALTH DENVER, LLC",License Issued - Active,70,BROADWAY,DENVER,80203.0
4,2014-BFN-1074231,SWEETWATER PARTNERS LLC,License Issued - Active,4260,KEARNEY,DENVER,80216.0


In [26]:
active_lic = active_lic.drop('Facility Street Number', axis=1)

In [28]:
active_lic.to_csv('den_cleaned_lic.csv')

In [30]:
crime_den.head()

,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,NEIGHBORHOOD_ID,date_occurred
0,weapon-unlawful-discharge-of,all-other-crimes,montbello,2016-06-15 23:31:00
1,theft-other,larceny,gateway-green-valley-ranch,2017-10-11 12:30:00
2,theft-items-from-vehicle,theft-from-motor-vehicle,wellshire,2016-03-04 20:00:00
3,theft-other,larceny,belcaro,2018-01-30 19:20:00
4,theft-shoplift,larceny,cherry-creek,2017-06-22 20:53:00


In [31]:
den_crime_dict = {'weapon-unlawful-discharge-of': 'other',
 'traf-other': 'other',
 'criminal-trespassing': 'other',
 'traffic-accident': 'other',
 'kidnap-adult-victim': 'other',
 'violation-of-restraining-order': 'other',
 'vehicular-eluding-no-chase': 'other',
 'contraband-into-prison': 'other',
 'weapon-poss-illegal-dangerous': 'other',
 'sex-off-fail-to-register': 'other',
 'weapon-by-prev-offender-powpo': 'other',
 'violation-of-court-order': 'other',
 'littering': 'other',
 'police-false-information': 'other',
 'police-resisting-arrest': 'other',
 'police-interference': 'other',
 'accessory-conspiracy-to-crime': 'other',
 'weapon-carrying-prohibited': 'other',
 'animal-cruelty-to': 'other',
 'bomb-threat': 'other',
 'public-fighting': 'other',
 'sex-off-registration-viol': 'other',
 'kidnap-dv': 'other',
 'pawn-broker-viol': 'other',
 'false-imprisonment': 'other',
 'weapon-other-viol': 'other',
 'eavesdropping': 'other',
 'probation-violation': 'other',
 'property-crimes-other': 'other',
 'weapon-carrying-concealed': 'other',
 'police-disobey-lawful-order': 'other',
 'fireworks-possession': 'other',
 'escape': 'other',
 'reckless-endangerment': 'other',
 'disarming-a-peace-officer': 'other',
 'wiretapping': 'other',
 'illegal-dumping': 'other',
 'police-obstruct-investigation': 'other',
 'extortion': 'other',
 'other-enviornment-animal-viol': 'other',
 'obstructing-govt-operation': 'other',
 'stolen-property-buy-sell-rec': 'other',
 'parole-violation': 'other',
 'weapon-flourishing': 'other',
 'gambling-gaming-operation': 'other',
 'contraband-possession': 'other',
 'intimidation-of-a-witness': 'other',
 'violation-of-custody-order': 'other',
 'vehicular-eluding': 'other',
 'explosive-incendiary-dev-pos': 'other',
 'explosive-incendiary-dev-use': 'other',
 'animal-poss-of-dangerous': 'other',
 'police-making-a-false-rpt': 'other',
 'gambling-device': 'other',
 'homicide-conspiracy': 'other',
 'bribery': 'other',
 'weapon-unlawful-sale': 'other',
 'failure-to-appear': 'other',
 'escape-aiding': 'other',
 'traf-impound-vehicle': 'other',
 'weapon-altering-serial-number': 'other',
 'explosives-posses': 'other',
 'money-laundering': 'other',
 'failure-to-report-abuse': 'other',
 'theft-other': 'larceny',
 'theft-shoplift': 'larceny',
 'theft-from-bldg': 'larceny',
 'theft-bicycle': 'larceny',
 'theft-of-services': 'larceny',
 'theft-unauth-use-of-ftd': 'larceny',
 'theft-from-mails': 'larceny',
 'theft-of-rental-property': 'larceny',
 'theft-purse-snatch-no-force': 'larceny',
 'theft-pick-pocket': 'larceny',
 'theft-gas-drive-off': 'larceny',
 'theft-from-yards': 'larceny',
 'theft-of-cable-services': 'larceny',
 'theft-items-from-vehicle': 'theft from motor vehicle',
 'theft-parts-from-vehicle': 'theft from motor vehicle',
 'burg-auto-theft-resd-w-force': 'theft from motor vehicle',
 'burg-auto-theft-resd-no-force': 'theft from motor vehicle',
 'burg-auto-theft-busn-w-force': 'theft from motor vehicle',
 'burg-auto-theft-busn-no-force': 'theft from motor vehicle',
 'drug-methampetamine-possess': 'drug-alc',
 'liquor-possession': 'drug-alc',
 'drug-synth-narcotic-possess': 'drug-alc',
 'drug-heroin-possess': 'drug-alc',
 'drug-poss-paraphernalia': 'drug-alc',
 'traffic-accident-dui-duid': 'drug-alc',
 'drug-cocaine-possess': 'drug-alc',
 'drug-heroin-sell': 'drug-alc',
 'drug-fraud-to-obtain': 'drug-alc',
 'drug-marijuana-possess': 'drug-alc',
 'drug-hallucinogen-possess': 'drug-alc',
 'drug-pcs-other-drug': 'drug-alc',
 'drug-methampetamine-sell': 'drug-alc',
 'drug-cocaine-sell': 'drug-alc',
 'drug-marijuana-cultivation': 'drug-alc',
 'drug-marijuana-sell': 'drug-alc',
 'drug-methamphetamine-mfr': 'drug-alc',
 'liquor-sell': 'drug-alc',
 'drug-synth-narcotic-sell': 'drug-alc',
 'drug-opium-or-deriv-possess': 'drug-alc',
 'drug-hallucinogen-mfr': 'drug-alc',
 'drug-hallucinogen-sell': 'drug-alc',
 'drug-make-sell-other-drug': 'drug-alc',
 'drug-opium-or-deriv-sell': 'drug-alc',
 'drug-barbiturate-possess': 'drug-alc',
 'drug-barbiturate-sell': 'drug-alc',
 'drug-barbiturate-mfr': 'drug-alc',
 'liquor-misrepresent-age-minor': 'drug-alc',
 'liquor-other-viol': 'drug-alc',
 'theft-of-motor-vehicle': 'auto theft',
 'robbery-car-jacking': 'auto theft',
 'theft-stln-veh-const-eqpt': 'auto theft',
 'theft-stln-vehicle-trailer': 'auto theft',
 'fraud-by-use-of-computer': 'white collar',
 'theft-embezzle': 'white collar',
 'forgery-other': 'white collar',
 'theft-fail-return-rent-veh': 'white collar',
 'forgery-counterfeit-of-obj': 'white collar',
 'fraud-identity-theft': 'white collar',
 'fraud-nsf-closed-account': 'white collar',
 'forgery-checks': 'white collar',
 'fraud-by-telephone': 'white collar',
 'fraud-criminal-impersonation': 'white collar',
 'forgery-poss-of-forged-inst': 'white collar',
 'drug-forgery-to-obtain': 'white collar',
 'impersonation-of-police': 'white collar',
 'theft-confidence-game': 'white collar',
 'gambling-betting-wagering': 'white collar',
 'forgery-poss-of-forged-ftd': 'white collar',
 'forgery-posses-forge-device': 'white collar',
 'altering-vin-number': 'white collar',
 'burglary-business-by-force': 'burglary',
 'burglary-residence-by-force': 'burglary',
 'burglary-business-no-force': 'burglary',
 'burglary-residence-no-force': 'burglary',
 'burglary-vending-machine': 'burglary',
 'burglary-poss-of-tools': 'burglary',
 'burglary-safe': 'burglary',
 'criminal-mischief-mtr-veh': 'pub disorder',
 'criminal-mischief-other': 'pub disorder',
 'public-order-crimes-other': 'pub disorder',
 'criminal-mischief-graffiti': 'pub disorder',
 'disturbing-the-peace': 'pub disorder',
 'indecent-exposure': 'pub disorder',
 'harassment': 'pub disorder',
 'harassment-sexual-in-nature': 'pub disorder',
 'public-peace-other': 'pub disorder',
 'curfew': 'pub disorder',
 'harassment-dv': 'pub disorder',
 'prostitution-engaging-in': 'pub disorder',
 'prostitution-aiding': 'pub disorder',
 'prostitution-pimping': 'pub disorder',
 'pub-peace-desecrate-symb': 'pub disorder',
 'loitering': 'pub disorder',
 'riot': 'pub disorder',
 'riot-incite': 'pub disorder',
 'aggravated-assault': 'agg assault',
 'traf-vehicular-assault': 'agg assault',
 'aggravated-assault-dv': 'agg assault',
 'weapon-fire-into-occ-bldg': 'agg assault',
 'agg-aslt-police-weapon': 'agg assault',
 'weapon-fire-into-occ-veh': 'agg assault',
 'aslt-agg-police-gun': 'agg assault',
 'traffic-accident-hit-and-run': 'other crimes against persons',
 'threats-to-injure': 'other crimes against persons',
 'assault-simple': 'other crimes against persons',
 'harassment-stalking-dv': 'other crimes against persons',
 'assault-police-simple': 'other crimes against persons',
 'assault-dv': 'other crimes against persons',
 'menacing-felony-w-weap': 'other crimes against persons',
 'obscene-material-possess': 'other crimes against persons',
 'obscene-material-mfr': 'other crimes against persons',
 'window-peeping': 'other crimes against persons',
 'robbery-street': 'robbery',
 'robbery-residence': 'robbery',
 'robbery-business': 'robbery',
 'robbery-bank': 'robbery',
 'robbery-purse-snatch-w-force': 'robbery',
 'sex-aslt-non-rape': 'sexual assault',
 'sex-aslt-rape': 'sexual assault',
 'sex-aslt-rape-pot': 'sexual assault',
 'sex-aslt-fondle-adult-victim': 'sexual assault',
 'sex-aslt-non-rape-pot': 'sexual assault',
 'sex-aslt-w-object': 'sexual assault',
 'sex-aslt-w-object-pot': 'sexual assault',
 'homicide-other': 'murder',
 'traf-vehicular-homicide': 'murder',
 'homicide-family': 'murder',
 'homicide-negligent': 'murder',
 'traf-habitual-offender': 'arson',
 'arson-other': 'arson',
 'arson-vehicle': 'arson',
 'arson-business': 'arson',
 'arson-residence': 'arson',
 'arson-public-building': 'arson'}

In [33]:
crime_den['offense_category'] = crime_den['OFFENSE_TYPE_ID'].apply(lambda x: den_crime_dict[x])
crime_den.head()

,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,NEIGHBORHOOD_ID,date_occurred,offense_category
0,weapon-unlawful-discharge-of,all-other-crimes,montbello,2016-06-15 23:31:00,other
1,theft-other,larceny,gateway-green-valley-ranch,2017-10-11 12:30:00,larceny
2,theft-items-from-vehicle,theft-from-motor-vehicle,wellshire,2016-03-04 20:00:00,theft from motor vehicle
3,theft-other,larceny,belcaro,2018-01-30 19:20:00,larceny
4,theft-shoplift,larceny,cherry-creek,2017-06-22 20:53:00,larceny


In [34]:
crime_den = crime_den.drop('OFFENSE_CATEGORY_ID',axis=1)

In [35]:
crime_den.head()

,OFFENSE_TYPE_ID,NEIGHBORHOOD_ID,date_occurred,offense_category
0,weapon-unlawful-discharge-of,montbello,2016-06-15 23:31:00,other
1,theft-other,gateway-green-valley-ranch,2017-10-11 12:30:00,larceny
2,theft-items-from-vehicle,wellshire,2016-03-04 20:00:00,theft from motor vehicle
3,theft-other,belcaro,2018-01-30 19:20:00,larceny
4,theft-shoplift,cherry-creek,2017-06-22 20:53:00,larceny


In [37]:
crime_den.to_csv('den_crime_squeaky.csv')

In [38]:
# denver crime and denver licenses both lood good!

In [41]:
df = pd.read_csv('data_cln/den_lic_clean.csv')
df.head()

,Unnamed: 0,Entity Name,Current License Status,Facility Street Name,Facility City,Facility Zip Code,year_issued
0,0,"HERBAL ALTERNATIVES, LLC",License Issued - Active,BROADWAY,DENVER,80210.0,2013
1,1,"GAIAS GARDEN, LLC",License Issued - Active,Elati,Denver,80216.0,2014
2,2,"PLATTE VALLEY DISPENSARY, LLC",License Issued - Active,BLAKE,DENVER,80205.0,2014
3,3,"ABUNDANT HEALTH DENVER, LLC",License Issued - Active,BROADWAY,DENVER,80203.0,2014
4,4,SWEETWATER PARTNERS LLC,License Issued - Active,KEARNEY,DENVER,80216.0,2014


In [42]:
den

AttributeError: '_Printer' object has no attribute 'head'

In [43]:
active_lic.head()

,Business File Number,Entity Name,Current License Status,Facility Street Name,Facility City,Facility Zip Code
0,2013-BFN-1070104,"HERBAL ALTERNATIVES, LLC",License Issued - Active,BROADWAY,DENVER,80210.0
1,2014-BFN-1072246,"GAIAS GARDEN, LLC",License Issued - Active,Elati,Denver,80216.0
2,2014-BFN-1072819,"PLATTE VALLEY DISPENSARY, LLC",License Issued - Active,BLAKE,DENVER,80205.0
3,2014-BFN-1073783,"ABUNDANT HEALTH DENVER, LLC",License Issued - Active,BROADWAY,DENVER,80203.0
4,2014-BFN-1074231,SWEETWATER PARTNERS LLC,License Issued - Active,KEARNEY,DENVER,80216.0
